# অধ্যায় ৭: চ্যাট অ্যাপ্লিকেশন তৈরি করা
## Github Models API দ্রুত শুরু

এই নোটবুকটি [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) থেকে নেওয়া হয়েছে, যেখানে [Azure OpenAI](notebook-azure-openai.ipynb) সার্ভিসে অ্যাক্সেস করার জন্য বিভিন্ন নোটবুক রয়েছে।


# সংক্ষিপ্ত বিবরণ  
"বড় ভাষা মডেলগুলো এমন ফাংশন যা টেক্সট থেকে টেক্সটে ম্যাপ করে। একটি ইনপুট টেক্সট স্ট্রিং দিলে, একটি বড় ভাষা মডেল পরবর্তী কোন টেক্সটটি আসবে তা অনুমান করার চেষ্টা করে"(১)। এই "কুইকস্টার্ট" নোটবুকটি ব্যবহারকারীদের উচ্চ-স্তরের LLM ধারণা, AML দিয়ে শুরু করার জন্য মূল প্যাকেজের প্রয়োজনীয়তা, প্রম্পট ডিজাইনের একটি সহজ পরিচিতি, এবং বিভিন্ন ব্যবহারের ক্ষেত্রে কয়েকটি সংক্ষিপ্ত উদাহরণ তুলে ধরবে।


## সূচিপত্র  

[সংক্ষিপ্ত বিবরণ](../../../../07-building-chat-applications/python)  
[কিভাবে OpenAI সার্ভিস ব্যবহার করবেন](../../../../07-building-chat-applications/python)  
[১. আপনার OpenAI সার্ভিস তৈরি করা](../../../../07-building-chat-applications/python)  
[২. ইনস্টলেশন](../../../../07-building-chat-applications/python)    
[৩. শংসাপত্র](../../../../07-building-chat-applications/python)  

[ব্যবহারিক ক্ষেত্রসমূহ](../../../../07-building-chat-applications/python)    
[১. টেক্সট সংক্ষেপণ](../../../../07-building-chat-applications/python)  
[২. টেক্সট শ্রেণিবিন্যাস](../../../../07-building-chat-applications/python)  
[৩. নতুন পণ্যের নাম তৈরি](../../../../07-building-chat-applications/python)  
[৪. একটি শ্রেণিবিন্যাসকারী ফাইন টিউন করা](../../../../07-building-chat-applications/python)  

[তথ্যসূত্র](../../../../07-building-chat-applications/python)


### আপনার প্রথম প্রম্পট তৈরি করুন  
এই সংক্ষিপ্ত অনুশীলনটি আপনাকে Github Models-এ একটি সাধারণ কাজ "সারাংশ তৈরি" করার জন্য মডেলে প্রম্পট জমা দেওয়ার মৌলিক ধারণা দেবে।

**ধাপসমূহ**:  
1. যদি এখনো না করে থাকেন, তাহলে আপনার পাইথন এনভায়রনমেন্টে `azure-ai-inference` লাইব্রেরি ইনস্টল করুন।  
2. স্ট্যান্ডার্ড হেল্পার লাইব্রেরি লোড করুন এবং Github Models-এর জন্য ক্রেডেনশিয়াল সেটআপ করুন।  
3. আপনার কাজের জন্য একটি মডেল নির্বাচন করুন  
4. মডেলের জন্য একটি সহজ প্রম্পট তৈরি করুন  
5. আপনার অনুরোধটি মডেল API-তে জমা দিন!


### 1. `azure-ai-inference` ইনস্টল করুন


In [ ]:
%pip install azure-ai-inference

### 2. সহায়ক লাইব্রেরি ইমপোর্ট করুন এবং শংসাপত্র ইনস্ট্যানশিয়েট করুন


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### ৩. সঠিক মডেল খুঁজে বের করা  
GPT-3.5-turbo বা GPT-4 মডেলগুলো প্রাকৃতিক ভাষা বুঝতে এবং তৈরি করতে পারে।


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## ৪. প্রম্পট ডিজাইন  

"বড় ভাষা মডেলগুলোর জাদু হলো, বিশাল পরিমাণ টেক্সটে পূর্বাভাসের ভুল কমানোর জন্য প্রশিক্ষিত হতে গিয়ে, এই মডেলগুলো এমন কিছু ধারণা শিখে ফেলে যা এই পূর্বাভাসের জন্য দরকারি। উদাহরণস্বরূপ, তারা নিচের বিষয়গুলো শিখে ফেলে"(১):

* কীভাবে বানান লিখতে হয়
* ব্যাকরণ কীভাবে কাজ করে
* কীভাবে বাক্য পুনর্লিখন করতে হয়
* কীভাবে প্রশ্নের উত্তর দিতে হয়
* কীভাবে কথোপকথন চালাতে হয়
* কীভাবে বিভিন্ন ভাষায় লিখতে হয়
* কীভাবে কোড লিখতে হয়
* ইত্যাদি

#### কীভাবে বড় ভাষা মডেল নিয়ন্ত্রণ করবেন  
"বড় ভাষা মডেলের সব ইনপুটের মধ্যে সবচেয়ে বেশি প্রভাব ফেলে টেক্সট প্রম্পট(১)।

বড় ভাষা মডেলগুলোকে কয়েকভাবে আউটপুট দিতে প্রম্পট করা যায়:

নির্দেশনা: মডেলকে বলুন আপনি কী চান  
সম্পূর্ণকরণ: আপনি যা চান তার শুরুটা দিয়ে মডেলকে শেষটা লিখতে উদ্বুদ্ধ করুন  
ডেমোনস্ট্রেশন: মডেলকে দেখান আপনি কী চান, হয়  
প্রম্পটে কয়েকটি উদাহরণ দিয়ে  
বা ফাইন-টিউনিং ট্রেনিং ডেটাসেটে শত শত বা হাজার হাজার উদাহরণ দিয়ে"



#### প্রম্পট তৈরির তিনটি মূল নির্দেশিকা আছে:

**দেখান এবং বলুন**। আপনি কী চান, সেটা স্পষ্টভাবে জানান, হয় নির্দেশনা দিয়ে, উদাহরণ দিয়ে, অথবা দুটো মিলিয়ে। যদি আপনি চান মডেল কোনো তালিকাকে বর্ণানুক্রমিকভাবে সাজাক বা কোনো অনুচ্ছেদকে অনুভূতির ভিত্তিতে শ্রেণিবদ্ধ করুক, তাহলে সেটাই দেখান।

**গুণগত মানসম্পন্ন ডেটা দিন**। আপনি যদি কোনো শ্রেণিবিন্যাসকারী তৈরি করতে চান বা মডেলকে কোনো প্যাটার্ন অনুসরণ করাতে চান, তাহলে যথেষ্ট উদাহরণ দিন। আপনার উদাহরণগুলো ভালোভাবে দেখে নিন — মডেল সাধারণত যথেষ্ট বুদ্ধিমান, তাই ছোটখাটো বানান ভুলও ধরতে পারে এবং উত্তর দিতে পারে, কিন্তু মডেল ধরে নিতে পারে এটা ইচ্ছাকৃত এবং এতে উত্তর প্রভাবিত হতে পারে।

**আপনার সেটিংস পরীক্ষা করুন।** temperature এবং top_p সেটিংস নিয়ন্ত্রণ করে মডেল কতটা নির্ধারিতভাবে উত্তর দেবে। যদি এমন কোনো প্রশ্ন করেন যার একটাই সঠিক উত্তর, তাহলে এগুলো কমিয়ে দিন। যদি বেশি বৈচিত্র্যময় উত্তর চান, তাহলে এগুলো বাড়াতে পারেন। এই সেটিংস নিয়ে সবচেয়ে বড় ভুল হলো, অনেকে ধরে নেন এগুলো "বুদ্ধিমত্তা" বা "সৃজনশীলতা" নিয়ন্ত্রণ করে।


সূত্র: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### একই কলটি পুনরাবৃত্তি করুন, ফলাফলগুলি কীভাবে তুলনা করা যায়?


In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## টেক্সট সংক্ষেপ করুন  
#### চ্যালেঞ্জ  
একটি টেক্সট প্যাসেজের শেষে 'tl;dr:' যোগ করে টেক্সট সংক্ষেপ করুন। লক্ষ্য করুন, মডেলটি কোনো অতিরিক্ত নির্দেশনা ছাড়াই কীভাবে বিভিন্ন কাজ করতে পারে তা বুঝতে পারে। আপনি tl;dr-এর চেয়ে আরও বর্ণনামূলক প্রম্পট ব্যবহার করে মডেলের আচরণ পরিবর্তন ও সংক্ষেপণের ধরন কাস্টমাইজ করতে পারেন(3)।

সাম্প্রতিক গবেষণায় দেখা গেছে, বড় টেক্সট কর্পাসে প্রি-ট্রেইনিং এবং তারপর নির্দিষ্ট টাস্কে ফাইন-টিউনিংয়ের মাধ্যমে অনেক NLP টাস্ক ও বেঞ্চমার্কে উল্লেখযোগ্য অগ্রগতি হয়েছে। যদিও এই পদ্ধতির আর্কিটেকচার সাধারণত টাস্ক-অ্যাগনস্টিক, তবুও এতে হাজার হাজার বা দশ হাজারেরও বেশি উদাহরণসহ টাস্ক-নির্দিষ্ট ফাইন-টিউনিং ডেটাসেট প্রয়োজন। এর বিপরীতে, মানুষ সাধারণত মাত্র কয়েকটি উদাহরণ বা সহজ নির্দেশনা থেকেই নতুন ভাষাগত কাজ করতে পারে—যা বর্তমান NLP সিস্টেমগুলো এখনো বেশিরভাগ ক্ষেত্রেই করতে পারে না। এখানে আমরা দেখিয়েছি, ভাষা মডেল বড় হলে টাস্ক-অ্যাগনস্টিক, ফিউ-শট পারফরম্যান্স অনেক উন্নত হয়, কখনো কখনো আগের সেরা ফাইন-টিউনিং পদ্ধতির সঙ্গে প্রতিযোগিতাও করতে পারে।



Tl;dr


# বিভিন্ন ব্যবহারের জন্য অনুশীলন  
1. টেক্সট সংক্ষেপণ করুন  
2. টেক্সট শ্রেণিবদ্ধ করুন  
3. নতুন পণ্যের নাম তৈরি করুন


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## টেক্সট শ্রেণীবিন্যাস করুন  
#### চ্যালেঞ্জ  
ইনফারেন্সের সময় দেওয়া ক্যাটেগরিগুলোর মধ্যে আইটেমগুলোকে শ্রেণীবিন্যাস করুন। নিচের উদাহরণে, আমরা প্রম্পটে (playground_reference) ক্যাটেগরি এবং শ্রেণীবিন্যাস করার টেক্সট দুটোই দিয়েছি।

গ্রাহকের জিজ্ঞাসা: হ্যালো, আমার ল্যাপটপের কীবোর্ডের একটি কি সম্প্রতি ভেঙে গেছে এবং আমাকে একটি রিপ্লেসমেন্ট লাগবে:

শ্রেণীবদ্ধ ক্যাটেগরি:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## নতুন পণ্যের নাম তৈরি করুন
#### চ্যালেঞ্জ
উদাহরণ শব্দ থেকে পণ্যের নাম তৈরি করুন। এখানে আমরা যে পণ্যের জন্য নাম তৈরি করতে যাচ্ছি, তার তথ্য প্রম্পটে যুক্ত করেছি। আমরা একটি অনুরূপ উদাহরণও দিয়েছি, যাতে কাঙ্ক্ষিত প্যাটার্ন বোঝানো যায়। আরও বেশি বৈচিত্র্য ও উদ্ভাবনী উত্তর পেতে আমরা temperature মান বেশি রেখেছি।

পণ্যের বিবরণ: একটি ঘরোয়া মিল্কশেক মেকার
বীজ শব্দ: দ্রুত, স্বাস্থ্যকর, কমপ্যাক্ট।
পণ্যের নাম: HomeShaker, Fit Shaker, QuickShake, Shake Maker

পণ্যের বিবরণ: এমন একজোড়া জুতো, যা যেকোনো পায়ের মাপে মানিয়ে যায়।
বীজ শব্দ: অভিযোজ্য, ফিট, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# রেফারেন্সসমূহ  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [টেক্সট শ্রেণিবিন্যাসের জন্য GPT-3 ফাইন-টিউনিংয়ের সেরা অনুশীলনসমূহ](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# আরও সাহায্যের জন্য  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# অবদানকারীরা
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুলতার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
